In [205]:
import pandas as pd
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
tv_movie_data = pd.read_csv("oneHotData.csv", converters={'desc_tokenized': eval, 'actor_tokenized': eval})
tv_movie_data.shape
N_CLUSTERS = 100

In [206]:
%pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import *

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chunhao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chunhao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chunhao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [207]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from ast import literal_eval
  
example_sent = """This is a sample sentence, 
                  showing off the stop words filtration."""
  
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
def superclean(x):
    x = [i.lower() for i in x]
    x = [stemmer.stem(i) for i in x]
    x = [i for i in x if i not in stop_words]
    x = [i for i in x if any(c.isalpha() for c in i)]
    return x
tv_movie_data['desc_tokenized'] = tv_movie_data['desc_tokenized'].apply(superclean)
tv_movie_data['actor_tokenized'] = tv_movie_data['actor_tokenized'].apply(superclean)
tv_movie_data.to_csv('new.csv')

In [210]:
tv_movie_data = pd.read_csv("new.csv")
sub = tv_movie_data
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(sub['actor_tokenized'])
vectorizer2 = TfidfVectorizer()
X2 = vectorizer2.fit_transform(sub['desc_tokenized'])
len(vectorizer2.get_feature_names())

53109

In [211]:
# OOM! BOOM!
# a = pd.DataFrame(X.toarray(), columns= [''.join(i) for i in zip(['w_'] * len(vectorizer.get_feature_names()), vectorizer.get_feature_names())])
# words = pd.DataFrame(X2.toarray(), columns= [''.join(i) for i in zip(['a_'] * len(vectorizer2.get_feature_names()), vectorizer2.get_feature_names())])

# Instead lets merge vectors into 1000 "macrovectors"
NUM_MACROVECTORS = 1000
step = X2.shape[1] // NUM_MACROVECTORS
x = X2.toarray()
res = np.empty((X2.shape[0], NUM_MACROVECTORS))
for i in range(NUM_MACROVECTORS):   
    if i < NUM_MACROVECTORS - 1:
        res[:,i] = np.sum(x[:, i * step: (i + 1) * step], axis=1)
    else:
        res[:,i] = np.sum(x[:, i * step:], axis=1)
print(res.shape)
words = pd.DataFrame(res, columns= [''.join(i) for i in zip(['d_'] * NUM_MACROVECTORS, [str(j) for j in range(NUM_MACROVECTORS)])])
words.head()

(82086, 1000)


,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_990,d_991,d_992,d_993,d_994,d_995,d_996,d_997,d_998,d_999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [212]:
DISCRIPTION_EFFECT= 0.2
test = pd.concat([sub, words * DISCRIPTION_EFFECT], axis=1)
unwanted = test.columns[test.columns.str.startswith('Unnamed')]
test = test.drop(unwanted, axis=1)
test = test.drop(['desc_tokenized','actor_tokenized', 'weighted_average_vote'], axis=1)
arr = test.drop(['imdb_name_id','imdb_title_id','title'], axis=1)

In [213]:
arr.head()
arr.to_csv('compressedvectors.csv')

,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,Film-Noir,...,d_990,d_991,d_992,d_993,d_994,d_995,d_996,d_997,d_998,d_999
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [216]:
kmeans = MiniBatchKMeans(n_clusters=N_CLUSTERS, random_state=0).fit(arr)
np.unique(kmeans.labels_, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([ 1643,   333,  1133, 13543,   763,  4109,   450,  2413,  1343,
         1452,  2952,  5276,   943,    50,   932,    59,   358,  1374,
          719,  1284,  2506,   483,  2273,   865,    70,   398,   397,
          281,   733,     1,   770,  1247,   629,   692,    12,   102,
            2,     1,   315,    18,     2,   396,   607,   110,   131,
          478,   420,  1878,  1143,   305,   467,   201,    96,   590,
           79,     1,   508,   276,   298,     1,   160,   601,   400,
          216,   118,   225,   364,   801,   513,   1

In [217]:
kmeans.labels_

array([28, 17,  3, ...,  3, 74,  3])

In [220]:
sub2 = pd.concat([sub, pd.Series(kmeans.labels_, name='kmeans')], axis=1)
sub2.to_csv('all_categories.csv')
for i in range(N_CLUSTERS):
    t = sub2[sub2['kmeans']==i]
    t.to_csv(str(i)+'.csv')

In [221]:
unwanted = sub2.columns[sub2.columns.str.startswith('Unnamed')]
sub2 = sub2.drop(unwanted, axis=1)
sub2 = sub2.set_index('imdb_title_id')
sub2.head()

,imdb_name_id,title,weighted_average_vote,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,desc_tokenized,actor_tokenized,kmeans
imdb_title_id,,,,,,,,,,,,,,,,,,,,,
tt0000009,nm0063086,Miss Jerry,5.9,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,"['adventur', 'femal', 'report']","['blanchebayliss', 'williamcourtenay', 'chaunc...",28
tt0000574,nm0170118,The Story of the Kelly Gang,6.1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,"['true', 'stori', 'notori', 'australian', 'out...","['elizabethtait', 'johntait', 'normancampbel',...",17
tt0001892,nm0003425,Den sorte drøm,5.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"['two', 'men', 'high', 'rank', 'woo', 'beauti'...","['astanielsen', 'valdemarpsiland', 'gunnarhels...",3
tt0002101,nm0182557,Cleopatra,5.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"['fabl', 'queen', 'egypt', ""'s"", 'affair', 'ro...","['helengardn', 'pearlsindelar', 'missfield', '...",33
tt0002130,nm0019604,L'Inferno,7.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,"['loos', 'adapt', 'dant', ""'s"", 'divin', 'come...","['salvatorepapa', 'arturopirovano', 'giusepped...",87


In [222]:
sub2.loc['tt0000009']['kmeans']

28

In [223]:
import csv
import urllib.parse
import urllib.request
import requests
import re
from bs4 import BeautifulSoup
import IPython.display as Disp
from tqdm import tqdm
import IPython.display as displ

def display(image_url, info=None): 
    if isinstance(image_url, str):
        image_url = [image_url]
    print('------------------------------------------------------')
    for index, i in enumerate(image_url):
        print('Result '+str(index) + ":")
        if info is None:
            info = image_url
        print(info[index])
        displ.display(displ.Image(url=i))
        print('------------------------------------------------------')

def get_poster_id(movie_title, exact=False, num_of_results=-1):
    flag = False
    if num_of_results == -1:
        flag = True
        num_of_results = 1
    domain = 'http://www.imdb.com'
    t = ''
    if exact:
        t = '&exact=true'
    search_url = domain + '/find?s=tt' + '&q=' + urllib.parse.quote_plus(movie_title)
    with urllib.request.urlopen(search_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        # Get url of 1st search result
        try:
            tables = soup.find('table', class_='findList').find_all('tr')[:num_of_results]
            movie_id = []
            for i in (tables):
                title = i.a['href']
                movie_id.append(title[title.rfind('/',0, title.rfind('/'))+1:-1])                   
        # Ignore cases where search returns no results
        except AttributeError:
            print('No search results.')  
            return '' if flag else ['']
    return movie_id[0] if flag else movie_id

def get_poster(movie_title, size=-1, exact=False, num_of_results=-1):
    flag = False
    if num_of_results == -1:
        flag = True
        num_of_results = 1
    domain = 'http://www.imdb.com'
    t = ''
    if exact:
        t = '&exact=true'
    search_url = domain + '/find?s=tt' + '&q=' + urllib.parse.quote_plus(movie_title)
    with urllib.request.urlopen(search_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        # Get url of 1st search result
        try:
            tables = soup.find('table', class_='findList').find_all('tr')[:num_of_results]
            images = []
            for i in (tables):
                image_url = i.img['src']
                if size==-1:
                    image_url = re.sub(r'(?<=._V1).*(?=\.jpg)','', image_url)
                else:
                    image_url = re.sub(r'(?<=._V1).*(?=\.jpg)','_UX' + str(size), image_url)
                images.append(image_url)                   
        # Ignore cases where search returns no results
        except AttributeError:
            print('No search results.')  
            return '' if flag else ['']
    return images[0] if flag else images

def get_poster_from_ids(movie_ids, size=-1):
    flag = False
    if isinstance(movie_ids, int):
        movie_ids = ['tt' + str(movie_ids)]
    elif isinstance(movie_ids, str):
        movie_ids = [movie_ids]
    elif isinstance(movie_ids, list):
        movie_ids = ['tt' + str(i) if isinstance(i, int) else i for i in movie_ids]
        flag = True
    res = []
    for movie_id in tqdm(movie_ids):
        movie_url = 'http://m.imdb.com/title/' + movie_id 
        with urllib.request.urlopen(movie_url) as response:
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            try:
                image_url = soup.find('div', class_='media titlemain__overview-media--mobile').a.img['src']
            except AttributeError:
                print('No poster for ' + movie_id)
                res.append('')
        if size==-1: 
            image_url = re.sub(r'(?<=._V1).*(?=\.jpg)','', image_url)
        else:
            image_url = re.sub(r'(?<=._V1).*(?=\.jpg)','_UX' + str(size), image_url)
        res.append(image_url)
    return res if flag else res[0]


In [231]:
def get_poster_id_interactive():
    movie_title = input('Please enter the name of the movie:').strip()
    ids = get_poster_id(movie_title, exact=False, num_of_results=5)
    display(get_poster(movie_title, size=100, exact=True, num_of_results=5))
    print()
    x = input('Please choose the movie you want:').strip()
    if x == '':
        x = 0
    else:
        x = int(x)
    id = ids[x]
    try:
        cat = sub2.loc[id]['kmeans']
    except Exception:
        print('It seems that it is a new movie. It is not in our local dataset.')
        return
    print('You choose {0}, which is in category {1}'.format(id, cat))
    others = sub2[sub2['kmeans']==cat].drop(id).sort_values(by='weighted_average_vote', ascending=False)[:10]
    ids = list(others.index)
    names = list(others['title'])
    print('You may also like:')
    display(get_poster_from_ids(ids, size=100), names)

In [235]:
get_poster_id_interactive()

Please enter the name of the movie:inception
------------------------------------------------------
Result 0:
https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_UX100.jpg


------------------------------------------------------
Result 1:
https://m.media-amazon.com/images/M/MV5BYWJmYWJmNWMtZTBmNy00M2MzLTg5ZWEtOGU5ZWRiYTE0ZjVmXkEyXkFqcGdeQXVyNzkyOTM2MjE@._V1_UX100.jpg


------------------------------------------------------
Result 2:
https://m.media-amazon.com/images/S/sash/85lhIiFCmSScRzu.png


------------------------------------------------------
Result 3:
https://m.media-amazon.com/images/M/MV5BMjE0NGIwM2EtZjQxZi00ZTE5LWExN2MtNDBlMjY1ZmZkYjU3XkEyXkFqcGdeQXVyNjMwNzk3Mjk@._V1_UX100.jpg


------------------------------------------------------
Result 4:
https://m.media-amazon.com/images/M/MV5BZGFjOTRiYjgtYjEzMS00ZjQ2LTkzY2YtOGQ0NDI2NTVjOGFmXkEyXkFqcGdeQXVyNDQ5MDYzMTk@._V1_UX100.jpg


...

------------------------------------------------------
Result 6:
Jurassic Park


------------------------------------------------------
Result 7:
Logan - The Wolverine


------------------------------------------------------
Result 8:
Terminator


------------------------------------------------------
Result 9:
The Avengers


------------------------------------------------------
